# Load Dataset

In [6]:
import datasets 
from datasets import load_dataset

dataset = load_dataset("hate_speech18")

In [7]:
# examine the label distribution
import pandas as pd
pd.Series(dataset['train']['label']).value_counts()

# filter out the ones with label 2 or 3
dataset['train'] = dataset['train'].filter(lambda example: example['label'] < 2)

# examine the label distribution again
pd.Series(dataset['train']['label']).value_counts()

0    9507
1    1196
Name: count, dtype: int64

In [9]:
# save the sampled dataset
dataset.save_to_disk('data/hatespeech')

Saving the dataset (1/1 shards): 100%|██████████| 10703/10703 [00:00<00:00, 129784.75 examples/s]


In [10]:
from utils import *
import llama
llama_config = ModelArguments()
llama_config.model = "meta-llama/Llama-2-7b-chat-hf"
llama_config.temperature = 0.3
llama_config.max_new_tokens = 100
llama_config.top_k = 5
llama_model = llama.LlamaModel(llama_config)

In [5]:
from inference_hatespeech import *
from tqdm import tqdm
correct,total = 0,0
bar = tqdm(range(len(dataset['train']['text'])))
with open('./llama_results/10000_sampled_hatespeech18_llama.txt', 'w+') as f:
    for i in bar:
        pred = llama_model.predict(PROMPT.format(dataset[i]['text']))
        pred = postprocess(pred[0])
        f.write(str(pred) + '\n')
        if pred == dataset[i]['label']:
            correct += 1
        total += 1
        bar.set_description(f"Accuracy: {correct/total:.4f}")

  0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy: 0.5884: 100%|██████████| 10000/10000 [4:44:39<00:00,  1.71s/it]  
